# 00-get-images-gee

Downloads Sentinel-2 (L2A), Sentinel-1, MODIS (MOD09A1), and topographic data for an area of interest and date coinciding with a flood event.

Sentinel-2 L2A data that intersects with the area of interest for a specified time-period before and after the flood event is retrieved, and, only images that have less than 10 % cloud cover within the area of interest are retained. MODIS (MOD09A1) data that coincides with the dates of cloud free Sentinel-2 data are retrieved. MODIS (MOD09A1) from the 12-day period before and after the flood event are also downloaded. This data is used to generate synthetic Sentinel-2 like data for pre- and post-flood images. The Sentinel-2 and MODIS green, red, and near infrared bands are downloaded.

The data is prepared using Google Earth Engine; therefore, a Google account with permissions to use Google Earth Engine's API is required to run this notebook.

In [ ]:
# import packages
import ee
import pprint
import os

In [ ]:
# earth engine authentication
ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
# S2Cloudless cloud probability threshold
CLD_PRB_THRESH = 50

def prep_samples(coords, event_date, gdrive_folder, gdrive_fname):
    """Generate and download ground truth data from GEE.
    Finds low and high resolution image pairs from two months before and
    one month after a flood event.

    Finds low resolution images from 12 days before to 48 days after a
    flood event.

    Finds Sentinel-1 and topography related features.


    Args:
        coords (list): GEE Polygon Geometry
        event_date (str): flood event date
        gdrive_folder (str): Google Drive folder to download images to
        gdrive_fname (str): filename id

    Returns:
        Download to Google Drive.
    """
    chip_coords = coords
    ee_event_date = ee.Date(event_date)

    pairs_months = ee.List.sequence(-2, 1)
    features_days = ee.List.sequence(-12, 36, 12)

    def make_pairs(m):
        """Map over months generating ImageCollections of monthly
        high and low resolution image pairs.

        Using Sentinel-2 Harmonized surface reflectance data
        and MODIS 8-Day 500 m composites.

        Find closest in-time 8-day composite of daily cloud free S2 images.

        Args:
            m (int): number of months pre- or post- event.

        Returns:
            ImageCollection: ImageCollection of cloud free S2 and MODIS pairs.
        """
        start = ee_event_date.advance(ee.Number(m), "month")
        end = ee_event_date.advance(ee.Number(m).add(1), "month")

        # get S2 collection
        L2A = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
            .filterDate(start, end) \
            .filterBounds(chip_coords) \
            .filter(ee.Filter.contains('.geo', chip_coords)) \
            .map(lambda image: image.clip(chip_coords))

        # also apply the cloud mask from the S2 Cloudless collection
        s2Cloudless = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY") \
            .filterDate(start, end) \
            .filterBounds(chip_coords) \
            .filter(ee.Filter.contains('.geo', chip_coords)) \
            .map(lambda image: image.clip(chip_coords))

        # join the S2 Cloudless collection to L2A
        s2CP = ee.ImageCollection(ee.Join.saveFirst("s2cloudless").apply(**{
            "primary": L2A,
            "secondary": s2Cloudless,
            "condition": ee.Filter.equals(**{
                "leftField": "system:index",
                "rightField": "system:index"
            })
        }))

        # add cloud probability as a band
        s2CP = s2CP.map(lambda image: image.addBands(
            ee.Image(image.get("s2cloudless"))))

        # use S2Cloudless to compute % of clear pixels in an image chip footprint.
        def cloud_mask_s2cloudless(image):
            # Get s2cloudless image, subset the probability band.
            # The s2cloudless probability layer is stored as a property of each image
            cld_prb = image.select("probability")
            proj = cld_prb.select("probability").projection()
            clouds_binary = cld_prb.gte(CLD_PRB_THRESH)

            # perform an opening operation
            # focalMin followed by focalMax
            clouds = clouds_binary.focalMin(**{
                "kernelType": "circle",
                "radius": 2}).reproject(
                proj, None, 10).rename("clouds")

            clouds = clouds.focalMax(**{
                "kernelType": "circle",
                "radius": 5}).reproject(
                proj, None, 10).rename("clouds")

            # number of pixels in image chip footprint
            all_count = clouds.reduceRegion(**{
                "reducer": ee.Reducer.count(),
                "geometry": clouds.geometry(),
                "scale": 10,
                "maxPixels": 1e13
            })

            # cloud free pixels (clouds are masked)
            tmp_is_not_cloud = clouds.eq(0).selfMask()

            # number of clear pixels in image chip footprint
            clear_count = tmp_is_not_cloud.reduceRegion(**{
                "reducer": ee.Reducer.count(),
                "geometry": tmp_is_not_cloud.geometry(),
                "scale": 10,
                "maxPixels": 1e13
            })

            # percentage of clear pixels in image chip footprint
            clear_percent = ee.Number(clear_count.get("clouds")) \
                .divide(ee.Number(all_count.get("clouds"))) \
                .multiply(100) \
                .round()

            return image.addBands(clouds).set({"clear_percent": clear_percent})

        # filter out only S2 images with  >90% clear pixels
        s2Masked = s2CP \
            .map(cloud_mask_s2cloudless) \
            .filter(ee.Filter.greaterThan("clear_percent", 90)) \
            .select(["B3", "B4", "B8"])

        return s2Masked

    # flatten list of ImageCollections to an ImageCollection
    s2Clear = ee.ImageCollection(
        ee.FeatureCollection(
            pairs_months.map(make_pairs)
        ).flatten()
    )

    # get MODIS 500 m bands corresponding to each cloud free S2 image
    def add_modis_bands(image):
        image_date = image.date()

        modis_500 = ee.ImageCollection("MODIS/061/MOD09A1") \
            .filter(ee.Filter.date(ee.Date(image_date).advance(-4, "day"), ee.Date(image_date).advance(5, "day"))) \
            .filterBounds(chip_coords) \
            .filter(ee.Filter.contains('.geo', chip_coords)) \
            .select(["sur_refl_b04", "sur_refl_b01", "sur_refl_b02"]) \
            .map(lambda image: image.resample("bicubic")) \
            .map(lambda image: image.clip(chip_coords)) \
            .sort("system:time_start", False) \
            .mosaic() \
            .select(["sur_refl_b04", "sur_refl_b01", "sur_refl_b02"]).rename(["B3_mod", "B4_mod", "B8_mod"])

        image = image.addBands(modis_500)

        return image

    # ImageCollection of clear S2 with MODIS bands attached
    s2Modis = s2Clear.map(add_modis_bands)

    s2reference = s2Modis.first().select(0).toInt16()

    # Download images of S2 with MODIS bands attached
    s2_modis_size = s2Modis.size().getInfo()
    download_s2_modis_list = s2Modis.toList(s2Modis.size())

    for i in range(0, s2_modis_size):
        download_image = ee.Image(
            download_s2_modis_list.get(i)).clip(chip_coords).cast(
            {"B3": "int16",
             "B4": "int16",
             "B8": "int16",
             "B3_mod": "int16",
             "B4_mod": "int16",
             "B8_mod": "int16"}
        )
        day = download_image.date().get("day").getInfo()
        month = download_image.date().get("month").getInfo()
        year = download_image.date().get("year").getInfo()
        clear_percent = download_image.get("clear_percent").getInfo()
        # include chip_idx to distinguish images on same date but different S2 tiles
        fname = f"low_high_pair_{gdrive_fname}_{year}_{month}_{day}_clearpct_{clear_percent}_chip_idx_{i}"

        ee.batch.Export.image.toDrive(
            image=download_image.clip(chip_coords),
            fileNamePrefix=fname,
            description=fname,
            folder=gdrive_folder,
            crs="EPSG:4326",
            skipEmptyTiles=True,
            scale=10,
            maxPixels=1e13).start()

    def make_coarse(day):
        """Map over 12 day chunks from 12 days before to 48 days after EMSR event.
        Return MODIS 500 m image for each 12-day chunk ensuring that all pixels are from retrievals within the chunk start and end days.

        Args:
            day (int):  start day of chunk to retrieve MODIS image. Starting 12 days before EMSR event.

        Returns:
            ImageCollection: ImageCollection of coarse MODIS images corresponding to each 12 day chunk.
        """
        start = ee_event_date.advance(ee.Number(day), "day")
        end = ee_event_date.advance(ee.Number(day).add(10), "day")
        start_doy = ee.Number.parse(start.format("DDD"))

        def doy_mask(image):
            # make sure all pixels in composite are from after start of time-slice
            doy_band = image.select("DayOfYear").clip(chip_coords)
            doy_band_mask = doy_band.gte(start_doy)
            doy_band_inv = doy_band.multiply(-1).rename(["quality"])
            image = image.addBands(doy_band_inv)

            return image.updateMask(doy_band_mask)

        modis_500 = ee.ImageCollection("MODIS/061/MOD09A1") \
            .filterDate(start, end) \
            .filterBounds(chip_coords) \
            .filter(ee.Filter.contains('.geo', chip_coords)) \
            .map(doy_mask) \
            .map(lambda image: image.resample("bicubic")) \
            .map(lambda image: image.clip(chip_coords)) \
            .qualityMosaic("quality") \
            .select(["sur_refl_b04", "sur_refl_b01", "sur_refl_b02"]).rename(["B3_mod", "B4_mod", "B8_mod"])

        return modis_500

    coarse_features = ee.ImageCollection(features_days.map(make_coarse))

    # Download coarse images
    coarse_features_size = coarse_features.size().getInfo()
    download_coarse_features_list = coarse_features.toList(
        coarse_features.size())

    for i in range(0, coarse_features_size):
        download_image = ee.Image(
            download_coarse_features_list.get(i)).clip(chip_coords).cast(
            {"B3_mod": "int16",
             "B4_mod": "int16",
             "B8_mod": "int16"}
        )

        fname = f"coarse_feature_{gdrive_fname}_{i}"

        ee.batch.Export.image.toDrive(
            image=download_image.clip(chip_coords),
            fileNamePrefix=fname,
            description=fname,
            folder=gdrive_folder,
            crs="EPSG:4326",
            skipEmptyTiles=True,
            scale=10,
            maxPixels=1e13).start()

    # Make Sentinel-1 features, topography features, and ground truth masks
    s1_post_start = ee_event_date.advance(0, "day")
    s1_post_end = ee_event_date.advance(12, "day")

    s1_pre_start = ee_event_date.advance(-24, "day")
    s1_pre_end = ee_event_date.advance(-12, "day")

    # function to add ratio band to s1 image
    def add_ratio_band(image):
        ratio_band = image.select("VV").divide(
            image.select("VH")).rename("VV/VH")

        return image.addBands(ratio_band)

    s1_post_event = ee.ImageCollection("COPERNICUS/S1_GRD") \
        .filter(ee.Filter.eq("instrumentMode", "IW")) \
        .filterDate(s1_post_start, s1_post_end) \
        .filterBounds(chip_coords) \
        .filter(ee.Filter.contains('.geo', chip_coords)) \
        .map(lambda image: image.clip(chip_coords)) \
        .map(add_ratio_band) \
        .sort("system:time_start", False) \
        .first() \
        .select(["VV", "VH", "VV/VH"]).rename(["VV_post", "VH_post", "VV/VH_post"])

    # get the orbit direction of the closed S1 image to the event data (either Descending or Ascending)
    # we cannot comapre ascending with descending
    # use this to get a suitable pre-event comparison image.
    post_orbit_direction = s1_post_event.get("orbitProperties_pass").getInfo()

    s1_pre_event = ee.ImageCollection("COPERNICUS/S1_GRD") \
        .filter(ee.Filter.eq("instrumentMode", "IW")) \
        .filter(ee.Filter.eq("orbitProperties_pass", post_orbit_direction)) \
        .filterDate(s1_pre_start, s1_pre_end) \
        .filterBounds(chip_coords) \
        .filter(ee.Filter.contains('.geo', chip_coords)) \
        .map(lambda image: image.clip(chip_coords)) \
        .map(add_ratio_band) \
        .sort("system:time_start", False) \
        .first() \
        .select(["VV", "VH", "VV/VH"]).rename(["VV_pre", "VH_pre", "VV/VH_pre"])

    # rescale to save int
    s1 = s1_post_event.addBands(s1_pre_event).multiply(10000).int32()

    ee.batch.Export.image.toDrive(
        image=s1.clip(chip_coords),
        fileNamePrefix=f"s1_{gdrive_fname}",
        description=f"s1_{gdrive_fname}",
        folder=gdrive_folder,
        crs="EPSG:4326",
        skipEmptyTiles=True,
        scale=10,
        maxPixels=1e13).start()

    # static features
    glo30 = ee.ImageCollection("projects/sat-io/open-datasets/GLO-30")
    dem = glo30.mosaic().setDefaultProjection(
        glo30.first().projection()).clip(chip_coords).multiply(10000).int16().rename("dem")
    slope = ee.Terrain.slope(dem).multiply(10000).int16().rename("slope")
    aspect = ee.Terrain.aspect(dem).multiply(10000).int16().rename("aspect")
    land_cover = ee.ImageCollection(
        "ESA/WorldCover/v100").first().clip(chip_coords).int16().rename("esa_world_cover")

    # get JRC permanent water as a flood susceptibility layer
    water = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')
    water = water.select(["occurrence"]).clip(chip_coords)

    # combine s1 and static
    topo_gt = s2reference.addBands(dem).addBands(slope).addBands(aspect).addBands(
        land_cover).addBands(water)

    ee.batch.Export.image.toDrive(
        image=topo_gt.clip(chip_coords),
        fileNamePrefix=f"topo_gt_{gdrive_fname}",
        description=f"topo_gt_{gdrive_fname}",
        folder=gdrive_folder,
        crs="EPSG:4326",
        skipEmptyTiles=True,
        scale=10,
        maxPixels=1e13).start()



In [ ]:
# Tropical Cyclone Yasa example
yasa_point = ee.Geometry.Point([179.41272700704917, -16.342607634743466])
yasa_buffer = yasa_point.buffer(2600)
yasa_chip = yasa_buffer.bounds()
event_date = "2020-12-17"
prep_samples(yasa_chip, event_date, "tc-yasa-aoi2", "fiji_yasa")